###### Load libraries and directories

In [1]:
from IPython.display import display, HTML

In [2]:
# from IPython import get_ipython
from tqdm.notebook import tqdm
import pickle
import os
import pprint
pp = pprint.PrettyPrinter(indent=1)

# Custom modules for debugging
from SliceViewer import ImageSliceViewer3D, ImageSliceViewer3D_1view,ImageSliceViewer3D_2views
from investigate import *

#pd.set_option("display.max_rows", 10)
      
import json
from run_sma_experiment import find_l3_images,output_images
import pprint
from L3_finder import *

# Custom functions
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):        
    with open(filename, 'rb') as input:
        return pickle.load(input)

Using TensorFlow backend.


In [3]:
0.22*216

47.52

In [4]:
get_ipython().run_line_magic('tb', '')

No traceback available to show.


In [5]:
cwd = os.getcwd()
print(cwd)
data = '/tf/data'
pickles = '/tf/pickles'
models = '/tf/models'

/tf/smipipeline


In [9]:
# Import modules and config file
configfile = os.path.join(cwd,'config/debug_ES/run_prediction_allsagreformats.json')
with open(configfile, "r") as f:
        config = json.load(f)
pp.pprint(config)

{'l3_finder': {'cache_dir': '/tf/_cache/',
               'cache_intermediate_results': True,
               'dicom_dir': '/tf/data',
               'model_path': '/tf/models/l3/child_9_slice_w_transfer_fold_3.h5',
               'new_tim_dicom_dir_structure': True,
               'output_directory': '/tf/output/all_sagittal_reformats/l3',
               'overwrite': True,
               'save_plots': True,
               'show_plots': False},
 'muscle_segmentor': {'model_path': '/tf/models/muscle/muscle_model_combined_dice_219_fold_1.h5',
                      'output_directory': '/tf/output/ms'}}


## Section 1 - Process final images

## Section 2 - Load each study into subject object
<br>
Subject object defined in L3finder.ingest

In [10]:
config['l3_finder']['new_tim_dicom_dir_structure']

True

In [11]:
# Debug
print("Finding subjects")

subjects = list(
    find_subjects(
        config['l3_finder']["dicom_dir"],
        new_tim_dir_structure=config['l3_finder']['new_tim_dicom_dir_structure']
    )
)

print('Subjects found: ', len(subjects))

Finding subjects
Subjects found:  2367


## Section-3 - check if there are subjects with multiple folders (studies)

In [12]:
%%time
# Find series images
print("Finding series")
series = list(flatten(tqdm((s.find_series() for s in subjects),total=len(subjects))))

Finding series



CPU times: user 392 ms, sys: 296 ms, total: 688 ms
Wall time: 1.68 s


In [13]:
print("Total number of series found: ", len(series))

Total number of series found:  3758


In [14]:
%%time
sagittal_series, axial_series, excluded_series = separate_series(series)

Filtering series
CPU times: user 820 ms, sys: 922 ms, total: 1.74 s
Wall time: 3.05 s


In [15]:
print("Length of valid pats: ", len(subjects))
print("Length of sagittal series", len(sagittal_series))
print("Length of axial series", len(axial_series))
print("Length of excluded series", len(excluded_series))
print("Length of all series in dataset", len(series))

Length of valid pats:  2367
Length of sagittal series 1391
Length of axial series 2367
Length of excluded series 0
Length of all series in dataset 3758


### Make sure each subject has at the max only 1 axial and 1 sagittal series

In [16]:
ax_ids = [ax.subject.id_ for ax in axial_series]
sag_ids = [sag.subject.id_ for sag in sagittal_series]

def find_duplicates(id_list):
    uniques = []
    duplicates = []
    for ids in id_list:
        if ids in uniques:
            duplicates.append(ids)
        else:
            uniques.append(ids)
            
    return uniques,duplicates


ax_u,ax_d = find_duplicates(ax_ids)
sag_u,sag_d = find_duplicates(sag_ids)

print('Ax duplicates: ', ax_d)
print('Sag duplicates: ', sag_d)

Ax duplicates:  []
Sag duplicates:  []


In [ ]:
# Find the series objects to investigate
ax_d_series = [ax for ax in axial_series if ax.subject.id_ in ax_d]
sag_d_series = [ax for ax in sagittal_series if ax.subject.id_ in ax_d]

In [ ]:
df_dl= load_object(os.path.join(pickles,'df_final.pkl'))
display(df_dl[df_dl['ID']==ax_d[0]])

## Reconstruct Missing Sagittals

In [17]:
sagittal_series_empty = []

In [18]:
# By default code filters 0.5mm slices, but I am letting them pass by setting it to 0
constructed_sagittals = construct_series_for_subjects_without_sagittals(
        subjects, sagittal_series_empty, axial_series,thickness_filter=0) 

FILTERING OUT 0.5 axials for recons for debugging!


In [19]:
print(
        "Series separated\n",
        len(sagittal_series), "sagittal series.",
        len(axial_series), "axial series.",
        len(excluded_series), "excluded series.",
        len(constructed_sagittals), "constructed series.",
    )

Series separated
 1391 sagittal series. 2367 axial series. 0 excluded series. 2367 constructed series.


In [21]:
# Logic to use all reconstructed sagittals or only for those that are missing sagittals
if len(sagittal_series) + len(constructed_sagittals) == len(axial_series):
    sagittal_series.extend(constructed_sagittals)
elif len(constructed_sagittals) == len(axial_series):
    sagittal_series = constructed_sagittals
else:
     print('Check series lengths, mismatch!')

In [22]:
save_object(axial_series,os.path.join(pickles,'axial_curated.pkl'))
save_object(sagittal_series,os.path.join(pickles,'sagittal_all_reformats_curated.pkl'))

## Create MIPS

In [25]:
print("Creating sagittal MIPS")
mips = create_sagittal_mips_from_series(
        many_series=sagittal_series,
        cache_dir=config['l3_finder'].get("cache_dir", None),
        cache=config['l3_finder'].get("cache_intermediate_results", False),
    )

Creating sagittal MIPS


100%|██████████| 2367/2367 [14:05<00:00,  2.80it/s]  


In [26]:
save_object(mips,os.path.join(pickles,'mips_all_reformats.pkl'))

## Find L3

In [ ]:
mips = load_object(os.path.join(pickles,'mips_all_reformats.pkl'))

In [27]:
print("Preprocessing Images")
preprocessed_images = preprocess_images(mips)

# Sagittal mip is redundant, get rid just use preprocessed images
sagittal_mips = [SagittalMIP(i) for i in preprocessed_images]

print("Separate heights for better batching")
mips_by_dimension = group_mips_by_dimension(sagittal_mips)
print("Dimensions in set:", mips_by_dimension.keys())

Preprocessing Images


0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py:611: UserWarning: From scipy 0.13.0, th

/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of 

Separate heights for better batching
Dimensions in set: dict_keys([(512, 512, 1), (1024, 512, 1)])


In [28]:
save_object(mips_by_dimension,os.path.join(pickles,'mips_by_dimension_alrf.pkl'))

## Find L3 - step 2

In [30]:
mips_by_dimension = load_object(os.path.join(pickles,'mips_by_dimension_alrf.pkl'))
axial_series = load_object(os.path.join(pickles,'axial_curated.pkl'))
sagittal_series = load_object(os.path.join(pickles,'sagittal_all_reformats_curated.pkl'))

In [31]:
print("Making predictions")
prediction_results = []
prediction_errors = []
for dimension, sagittal_mips in mips_by_dimension.items():
    dim_group_results,errors = make_predictions_for_sagittal_mips(
        sagittal_mips,
        model_path=config['l3_finder']["model_path"],
        shape=dimension
    )
    prediction_results.extend(dim_group_results)
    prediction_errors.extend(errors)

Making predictions


error drawing line on preprocessed_image for: Z892856
shape: (430, 512, 1), prediction: 431

error drawing line on preprocessed_image for: Z878452
shape: (450, 512, 1), prediction: 451

error drawing line on preprocessed_image for: Z1996094
shape: (125, 512, 1), prediction: 126

error drawing line on preprocessed_image for: Z1067609
shape: (290, 512, 1), prediction: 291

error drawing line on preprocessed_image for: Z370830
shape: (385, 512, 1), prediction: 386

error drawing line on preprocessed_image for: Z1140510
shape: (245, 512, 1), prediction: 246

error drawing line on preprocessed_image for: Z701477
shape: (185, 512, 1), prediction: 186

error drawing line on preprocessed_image for: Z495526
shape: (215, 512, 1), prediction: 216

error drawing line on preprocessed_image for: Z1119985
shape: (400, 512, 1), prediction: 401

error drawing line on preprocessed_image for: Z467063
shape: (425, 512, 1), prediction: 426

error drawing line on preprocessed_image for: Z878416
shape: (462,

In [32]:
print('Length of Prediction Results: ', len(prediction_results))

Length of Prediction Results:  2341


In [40]:
print('Length of Prediction Errors: ', len(prediction_errors))

Length of Prediction Errors:  26


In [34]:
save_object(prediction_results,os.path.join(pickles,'prediction_results_allrf.pkl'))
save_object(prediction_errors,os.path.join(pickles,'prediction_errors_allrf.pkl'))

## Load L3 Results

In [ ]:
prediction_errors = load_object(os.path.join(pickles,'prediction_errors_allrf.pkl'))

In [35]:
len(prediction_errors)

26

In [ ]:
prediction_results = load_object(os.path.join(pickles,'prediction_results_allrf.pkl'))
axial_series = load_object(os.path.join(pickles,'axial_curated.pkl'))

In [37]:
print('Building L3 images')
l3_images = build_l3_images(axial_series, prediction_results)

Building L3 images


In [38]:
# Don't run this unless you have new L3 results
print("Outputting L3 images")
# Clears pixel data from memory aafter outputting
l3_images = output_images(
    l3_images,
    args=dict(
        output_directory=config["l3_finder"]["output_directory"],
        should_plot=config["l3_finder"]["show_plots"],
        should_overwrite=config["l3_finder"]["overwrite"],
        should_save_plots=config["l3_finder"]["save_plots"]
    )
)

Outputting L3 images
Slow unless axial images already loaded...


100%|██████████| 2341/2341 [27:05<00:00,  1.44it/s] 


In [45]:
from run_sma_experiment import segment_muscle, calculate_smas,output_sma_results

In [46]:
print("Segmenting muscle...")
sma_images = segment_muscle(config["muscle_segmentor"], l3_images)

Segmenting muscle...
- Loading l3 axial images


2341it [01:14, 31.56it/s]
  5%|▌         | 126/2341 [00:00<00:01, 1259.13it/s]

- Removing table
 - Taking care of images that have a rescale value of -1024, as opposed to -2048 for cannon images


  6%|▋         | 151/2341 [00:00<00:01, 1505.08it/s]

  - zeroing images


100%|██████████| 2341/2341 [00:01<00:00, 1615.90it/s]

  - removing table



2341it [00:18, 128.19it/s]


- Thresholding images
- Normalizing images
- Resizing images
- Predicting segmentation
- Converting masks to images segmentation


In [47]:
print("Calculating sma")
areas = calculate_smas(sma_images)

Calculating sma


In [49]:
config["muscle_segmentor"]["output_directory"]

'/tf/output/ms'

In [48]:
print("Outputing SMA results")
output_sma_results(
    config["muscle_segmentor"]["output_directory"],
    sma_images,
    areas,
    [],
)

Outputing SMA results
